## Model

<img src="chart/image/plot.png" style="zoom:100%"/>

In [1]:
import os, shutil, json, cv2, glob, base64

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection, Line3DCollection
plt.rcParams['font.sans-serif'] = ['Taipei Sans TC Beta']

def plot(items, plate, size, gap, z_limit, path):
    fig = plt.figure(figsize=(12, 12))
    ax = fig.add_subplot(111, projection='3d') # 此ax為1x1網格中的第一個子圖，此圖為3D圖
    ax.view_init(elev = 90, azim = -90) # 調整視角

    for num, item in enumerate(items):
        # 定義立方體的八個頂點
        vertices = np.array([
            [item["x"]             , item["y"]             , item["z"]             ], # point0
            [item["x"] + item["dx"], item["y"]             , item["z"]             ], # point1
            [item["x"] + item["dx"], item["y"] + item["dy"], item["z"]             ], # point2
            [item["x"]             , item["y"] + item["dy"], item["z"]             ], # point3
            [item["x"]             , item["y"]             , item["z"] + item["dz"]], # point4
            [item["x"] + item["dx"], item["y"]             , item["z"] + item["dz"]], # point5
            [item["x"] + item["dx"], item["y"] + item["dy"], item["z"] + item["dz"]], # point6
            [item["x"]             , item["y"] + item["dy"], item["z"] + item["dz"]], # point7
            ])

        # 定義立方體的所有邊
        edges = [
            [0, 1], [1, 2], [2, 3], [3, 0], # 底邊
            [4, 5], [5, 6], [6, 7], [7, 4], # 頂邊
            [0, 4], [1, 5], [2, 6], [3, 7], # 四邊
            ]
        
        # 定義立方體的六個面
        faces = np.array([
            [0, 1, 2, 3], # 底面
            [0, 1, 5, 4],
            [1, 2, 6, 5],
            [2, 3, 7, 6],
            [3, 0, 4, 7],
            [4, 5, 6, 7], # 頂面
                        ])

        # 建立 Line3DCollection 物件，設置顏色和線條寬度
        lines = Line3DCollection([vertices[edge] for edge in edges], colors = 'black', linewidths = 0.5, alpha = 0.15)

        # 建立 Poly3DCollection 物件，設置顏色和透明度
        if num == 0: # 棧板顏色
            color = "#994D4D"
        elif num == len(items) - 1: # 新增的立方體設定藍色
            color = ["#D2E9FF", "#C4E1FF", "#C4E1FF", "#C4E1FF", "#C4E1FF", "#D2E9FF"] # 新的立方體
        else:
            color = ["#F4FFFF", "#EBFFFF", "#EBFFFF", "#EBFFFF", "#EBFFFF", "#F4FFFF"] # 舊的立方體
        

        cube = Poly3DCollection([vertices[faces[i]] for i in range(len(faces))], alpha = 0.8, facecolor = color)

        # 添加 Poly3DCollection 物件到坐標軸
        ax.add_collection3d(cube)
        ax.add_collection3d(lines) 

    # 在頂面添加文字描述
    ax.text(
        x = item["x"] + item["dx"]/2,
        y = item["y"] + item["dy"]/2,
        z = item["z"] + item["dz"],
        s = str(num), fontsize = 16, color = 'black', ha = 'center', va = 'center', alpha = 1, zorder = np.inf
        )

    # 設定標題和副標題
    ax.set_title(
        f'''棧板: {plate[0]}x{plate[1]}x{plate[2]},  箱子: {size[0]}x{size[1]}x{size[2]},  空隙: {gap}\n'''
        , fontsize = 20, color = 'black', ha = 'center', va = 'center'
        )
    
    # 添加解釋文字
    fig.text(
        x = 0.5, # 水平位置為圖片寬度的一半
        y = 0.1, # 垂直位置為圖片下方 5% 的位置
        s = f'''No. {num}
{item["info"]}
(x, y, z) = ({item["x"]}, {item["y"]}, {item["z"]})
(dx, dy, dz) = ({item["dx"]}, {item["dy"]}, {item["dz"]})''', # 解釋文字
        fontsize = 18, color = 'black', ha = 'center', va = 'center'
    )

    # 設置坐標軸範圍和標籤
    ax.set_xlim(0, plate[0])
    ax.set_ylim(0, plate[1])
    ax.set_zlim(0, z_limit)
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    
    ax.set_zticks([]) # z軸不顯示刻度

    # 設定坐標軸的比例，使其看起來更拉長
    ax.set_box_aspect([1, 1, 1])
    
    # 儲存圖片
    plt.savefig(os.path.join(path, f"{num}.png"))

    # 顯示圖形
    # plt.show()
    
    plt.close()

In [3]:
def set_position(point, dx, dy, dz):
    # 立方體座標，及移至左上角所需移動的量
    position = {
        "x": point[0], 
        "y": point[1], 
        "z": point[2],
        "dx": dx, 
        "dy": dy, 
        "dz": dz
    }

    return position

In [4]:
def calculate_counts(start, limit, length, width, even):
    x_range = limit[0] - start[0] # x軸可放立方體的空間
    y_range = limit[1] - start[1] # y軸可放立方體的空間

    total = 0
    flag = 1

    if ((length + width) <= x_range) and ((length + width) <= y_range): # 確認4個area至少都可以放下一個立方體
        x_count = (x_range - length) // width # 沿著x軸方向排列之立方體數量
        y_count = (y_range - length) // width # 沿著y軸方向排列之立方體數量

        while ((x_count * width) > ((length + (x_count * width)) / 2)) and ((y_count * width) < ((length + (y_count * width)) / 2)): # 處理重疊的部分 # 內層刪除多餘空間
        # while ((x_count * width) > (x_range / 2)) and ((y_count * width) < (y_range / 2)) and (length > y_range): # 處理重疊的部分 # 內層不刪除多餘空間
            x_count -= 1
        
        total = (x_count * 2) + (y_count * 2)
    else:
        x_count = y_count = 0
        counts = [0, 0, 0, 0]
    
    # 無法排四個區域，但可集中放在一個區域，即全部橫排(沿x軸排)或直排(沿y軸排)便可放入
    if (length <= y_range): # 橫排(沿x軸排)
        x_count1 = x_range // width # 沿著x軸方向排列之立方體數量
        if x_count1 > total:
            x_count = total = x_count1
            y_count = flag = 0
    if (length <= x_range): # 直排(沿y軸排)
        y_count1 = y_range // width # 沿著y軸方向排列之立方體數量
        if y_count1 > total:
            y_count = y_count1
            x_count = flag = 0
    
    if even:
        counts = [(y_count * flag), (x_count * flag), y_count, x_count]
    else:
        counts = [(x_count * flag), (y_count * flag), x_count, y_count]
    
    return x_count, y_count, counts

In [5]:
def add_item(area, points, length, width, catch, even):
    if even: # 雙數層
        if area == 0:
            cube = set_position(point = points[0], dx = length, dy = width, dz = catch) # 將立方體資訊儲存在dict中
            points[0][1] += width # 更新該區域立方體座標
        elif area == 1:
            cube = set_position(point = points[1], dx = -width, dy = length, dz = catch)
            points[1][0] -= width
        elif area == 2:
            cube = set_position(point = points[2], dx = -length, dy = -width, dz = catch)
            points[2][1] -= width
        else:
            cube = set_position(point = points[3], dx = width, dy = -length, dz = catch)
            points[3][0] += width
    else: # 單數層轉向排序
        if area == 0:
            cube = set_position(point = points[0], dx = width, dy = length, dz = catch)
            points[0][0] += width
        elif area == 1:
            cube = set_position(point = points[1], dx = -length, dy = width, dz = catch)
            points[1][1] += width
        elif area == 2:
            cube = set_position(point = points[2], dx = -width, dy = -length, dz = catch)
            points[2][0] -= width
        else:
            cube = set_position(point = points[3], dx = length, dy = -width, dz = catch)
            points[3][1] -= width
    
    return cube, points

In [6]:
def set_points(start, limit):
    point0 = [start[0], start[1], start[2]]
    point1 = [limit[0], start[1], start[2]]
    point2 = [limit[0], limit[1], start[2]]
    point3 = [start[0], limit[1], start[2]]

    points = [point0, point1, point2, point3]

    return points

In [7]:
def update_start_limit(start, limit, length, gap):
    start = [start[0] + (length + gap), start[1] + (length + gap), start[2]] # 更新點0
    limit = [limit[0] - (length - gap), limit[1] - (length - gap), start[2]] # 更新點2

    return start, limit

In [8]:
def sort_items(plate, items):
    total_items = []

    # 新增棧板位置
    total_items.append({"x": 0, "y": 0, "z": 0, "dx": plate[0], "dy": plate[1], "dz": plate[2], "info": "棧板"})

    # 新增立方體
    for layer_name, layer in items.items():
        areas = ["area3", "area2", "area1", "area0"] if (int(layer_name[5:]) % 2) == 0 else ["area2", "area1", "area3", "area0"] # 由外而內(原點)排序區域
        for i, area_name in enumerate(areas):
            if i >= 2:
                layer = dict(sorted(layer.items(), key = lambda x: x[0], reverse = True)) # 靠近原點的區域從內圈開始往外排，才能由外而內(原點)排序

            for lap in layer.values():
                if (((int(layer_name[5:]) % 2) == 0) and (area_name in ["area0"])) or\
                   (((int(layer_name[5:]) % 2) != 0) and (area_name in ["area1", "area0"])):
                    total_items.extend(lap[area_name][::-1]) # 倒序才能由外而內(原點)排序立方體
                else:
                    total_items.extend(lap[area_name])
    
    return total_items

In [9]:
def images_to_vedio(root, image_folder_path, param, hight, gap):
    # 設定輸出影片路徑
    folder_name = "_".join(str(i) for i in param["item"])
    folder_path = os.path.join(root, "data", "vedio", folder_name)

    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        
    output_path = os.path.join(folder_path, f"{hight}_{gap}.mp4")

    # 取得資料夾中所有的png檔案名稱，依照創建日期排序
    file_names = sorted(glob.glob(os.path.join(image_folder_path, "*.png")), key = os.path.getctime)

    # 讀取第一張圖片，並取得圖片大小
    first_frame = cv2.imread(file_names[0])
    frame_height, frame_width, _ = first_frame.shape

    # 設定影片編碼器和fps
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = 5

    # 建立影片寫入器
    video_writer = cv2.VideoWriter(filename = output_path, fourcc = fourcc, fps = fps, frameSize = (frame_width, frame_height))

    # 讀取每張圖片，並將圖片加入影片寫入器
    for file_name in file_names:
        img = cv2.imread(file_name) 
        video_writer.write(img)

    # 釋放影片寫入器
    video_writer.release()

In [10]:
def main(param):
    # 初始化存放圖片的資料夾
    root = param["root"]
    folder = os.path.join(root, "data", "image")
    if os.path.exists(folder):
        shutil.rmtree(folder) # 刪除資料夾

    # 棧板大小、總高、機器手臂空隙設定
    plate = [param["plate"][0], param["plate"][1], param["plate"][2]]
    z_limit = param["total_height"]
    gap = param["gap"]

    # 兩種排序方式: 長邊橫著牌或直著排
    options = [
        [param["item"][1], param["item"][2], param["item"][0]],
        [param["item"][0], param["item"][2], param["item"][1]]
        ]
    if param["item"][0] == param["item"][1]: # 如果長寬一樣，只有一種排法
        options.pop()
        print(options)

    # 依序計算不同擺放方式所能放入的總立方體數及位置
    datas = {}
    for option in options: 
        # 設定立方體的擺放方式
        length = option[0]
        width = option[1]
        hight = option[2]
        length_gap = length + gap # 長加上機器手臂的距離

        # 計算各層各圈各區域的立方體的座標及移動位置
        items = {}
        counts_info = {}
        layers = (z_limit - plate[2]) // hight # 計算共可疊幾層
        for layer in range(layers):
            # print(f"layer{layer}:")
            items[f"layer{layer}"] = {}
            
            start = [0, 0, (plate[2] + hight * layer)]# 各層的高須先加上棧板高度
            limit = [plate[0], plate[1], z_limit]

            laps = min(int(limit[0] // (length_gap * 2) + 1), int(limit[1] // (length_gap * 2) + 1)) # 計算該層共可放幾圈
            for lap in range(laps):
                # print(f"-> lap{lap}:")
                items[f"layer{layer}"][f"lap{lap}"] = {}

                even = (layer % 2 == 0) # True: 第0、2、4...層；False:第1、3、5...層
                x_count, y_count, counts = calculate_counts(start, limit, length_gap, width, even) # 計算該層該圈的各區域立方體數量

                # 記錄各層x、y軸擺放的立方體數量
                if layer == 0:
                    counts_info[f"lap{lap}"] = {"x_count": x_count, "y_count": y_count}

                # 刪除剩餘空間，即所有立方體往原點靠近，空隙往右上移
                # if (lap == 0):
                limit[0] = start[0] + (x_count * width) + (length_gap * (y_count != 0))
                limit[1] = start[1] + (y_count * width) + (length_gap * (x_count != 0))

                points = set_points(start, limit) # 初始化各區域立方體座標

                for area in range(4):
                    # print(f"--> area{area}")
                    items[f"layer{layer}"][f"lap{lap}"][f"area{area}"] = []
                    for _ in range(int(counts[area])):
                        cube, points = add_item(area, points, length, width, hight, even) # 新增立方體
                        items[f"layer{layer}"][f"lap{lap}"][f"area{area}"].append(cube) # 保存立方體資訊
                        
                        cube["info"] = f"第{layer + 1}層_第{area + 1}區_第{lap + 1}圈"

                start, limit = update_start_limit(start, limit, length, gap) # 更新下一圈的start和limit座標

        # 排序立方體，並將其放入list中以畫圖
        total_items = sort_items(plate, items)

        counts = len(total_items) - 1
        print(f"length: {length}, width: {width}, hight: {hight}, counts = {counts}")

        size = [length, width, hight]
        datas[hight] = {
            "size": size,
            "counts": counts,
            "layer": layers,
            "laps": laps,
            "total_hight": hight * (layer + 1) + plate[2], # 加棧板總高
            "layer_counts": int(counts / 2), # 每層有幾個
            "counts_info": counts_info, # 各圈x、y軸排法
            "data": items,
            "sort": total_items
        }

        if counts != 0:
            # 創建儲存圖片的資料夾
            image_folder_path = os.path.join(root, "data", "image", str(hight))
            os.makedirs(image_folder_path) 

            # 依序畫出各步驟的圖
            for i in range(1, len(total_items) + 1):
                plot(total_items[:i], plate, size, gap, z_limit, image_folder_path)
            
            # 將圖片轉為影片
            images_to_vedio(root, image_folder_path, param, hight, gap)
        
    # 儲存datas
    with open(os.path.join(root, "data", "data.json"), 'w') as f:
        json.dump(datas, f, indent = 4)

## Parameters

In [11]:
samples = [
    {
    "plate": [1110, 950, 100],
    "item": [605, 305, 86],
    "total_height": 1050,
    "gap": 20,
    "root" : os.getcwd()
    }, # 54 = 54
    {
    "plate": [1110, 950, 100],
    "item": [605, 605, 45],
    "total_height": 1315,
    "gap": 30,
    "root" : os.getcwd()
    }, # 60 < 68
    {
    "plate": [1110, 950, 100],
    "item": [905, 455, 40],
    "total_height": 1010,
    "gap": 15,
    "root" : os.getcwd()
    }, # 54 < 56
    {
    "plate": [1110, 950, 100],
    "item": [1205, 603, 30],
    "total_height": 1315,
    "gap": 15,
    "root" : os.getcwd()
    }, # 52 < 54
    {
    "plate": [1110, 950, 100],
    "item": [600, 140, 90],
    "total_height": 1315,
    "gap": 25,
    "root" : os.getcwd()
    }
]
sample_num = 4
param = samples[sample_num]

## dict encode to base64

In [12]:
s = json.dumps(param)
b = s.encode("UTF-8")
e = base64.b64encode(b)
param = e.decode("UTF-8")
print(param)

eyJwbGF0ZSI6IFsxMTEwLCA5NTAsIDEwMF0sICJpdGVtIjogWzYwMCwgMTQwLCA5MF0sICJ0b3RhbF9oZWlnaHQiOiAxMzE1LCAiZ2FwIjogMjUsICJyb290IjogImM6XFxVc2Vyc1xcdHp1bGlcXERvY3VtZW50c1xccHl0aG9uXFxwYWNraW5nIn0=


## base64 decode to dict

In [13]:
param = base64.b64decode(param).decode('utf-8')
param = json.loads(param)
param

{'plate': [1110, 950, 100],
 'item': [600, 140, 90],
 'total_height': 1315,
 'gap': 25,
 'root': 'c:\\Users\\tzuli\\Documents\\python\\packing'}

## Call model

In [14]:
# from run import *

main(param)

length: 140, width: 90, hight: 600, counts = 120
length: 600, width: 90, hight: 140, counts = 128
